<a href="https://colab.research.google.com/github/aviraltyagi/DeepLearningArchitectures/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, Input, MaxPool2D, Layer
import typing

In [ ]:
class CustomLayer(Layer):
  def __init__(self, filter, ranges: int):
    super(CustomLayer, self).__init__()
    self.Conv = Conv2D(filters= filter, kernel_size= (3,3), padding= 'same', activation= 'relu')
    self.Pool = MaxPool2D(strides= (2,2))
    self.r = range(ranges)
  
  def call(self, input, isPool = True, training = False):
    if isPool:
      input = self.Pool(input)
    for i in self.r:
      x = self.Conv(input, training = training)
    return x

In [ ]:
class CustomVGG16(Model):
  def VGGNet(input_shape: typing.Tuple[int], classes: int) -> Model:
    x_input = Input(shape= input_shape)

    # Layer 1
    x = CustomLayer(filter=64, ranges = 2)(x_input, isPool= False, training = True)

    # Layer 2
    x = CustomLayer(filter=128, ranges = 2)(x, training = True)

    # Layer 3
    x = CustomLayer(filter=256, ranges = 3)(x, training = True)

    # Layer 4
    x = CustomLayer(filter=512, ranges = 3)(x, training = True)

    # Layer 5
    x = CustomLayer(filter=512, ranges = 2)(x, training = True)

    x = MaxPool2D(strides= (2,2))(x)

    x = Flatten()(x)

    x = Dense(units= 4096, activation= 'relu')(x)
    x = Dense(units= 4096, activation= 'relu')(x)

    output = Dense(units= 1000, activation= 'softmax')(x)

    model = Model(inputs = x_input, outputs= output)

    return model

In [ ]:
model = CustomVGG16.VGGNet(input_shape= (224, 224, 3), classes= 10)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
custom_layer (CustomLayer)   (None, 224, 224, 64)      1792      
_________________________________________________________________
custom_layer_1 (CustomLayer) (None, 112, 112, 128)     73856     
_________________________________________________________________
custom_layer_2 (CustomLayer) (None, 56, 56, 256)       295168    
_________________________________________________________________
custom_layer_3 (CustomLayer) (None, 28, 28, 512)       1180160   
_________________________________________________________________
custom_layer_4 (CustomLayer) (None, 14, 14, 512)       2359808   
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 512)         0     